In [372]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression

In [168]:
X_train = np.load('C:/Users/Arcry/SkillBox/Test_ctrl2go/X_train.npy')
y_train = np.load('C:/Users/Arcry/SkillBox/Test_ctrl2go/y_train.npy')

X_test = np.load('C:/Users/Arcry/SkillBox/Test_ctrl2go/X_test.npy')
y_test = np.load('C:/Users/Arcry/SkillBox/Test_ctrl2go/y_test.npy')

In [137]:
y_train.shape

(7352, 6)

In [43]:
X_train.shape

(7352, 128, 9)

In [19]:
m,n,r = X_train.shape
out_arr = np.column_stack((np.repeat(np.arange(m),n),X_train.reshape(m*n,-1)))
out_df = pd.DataFrame(out_arr)

In [258]:
out_df.head(129)

,0,1,2,3,4,5,6,7,8,9
0,0.0,1.012817,-0.123217,0.102934,0.000181,0.010767,0.055561,0.030191,0.066014,0.022859
1,0.0,1.022833,-0.126876,0.105687,0.010139,0.006579,0.055125,0.043711,0.042699,0.010316
2,0.0,1.022028,-0.124004,0.102102,0.009276,0.008929,0.048405,0.035688,0.074850,0.013250
3,0.0,1.017877,-0.124928,0.106553,0.005066,0.007489,0.049775,0.040402,0.057320,0.017751
4,0.0,1.023680,-0.125767,0.102814,0.010810,0.006141,0.043013,0.047097,0.052343,0.002553
...,...,...,...,...,...,...,...,...,...,...
124,0.0,1.019290,-0.126185,0.098350,-0.000300,-0.002023,0.000359,0.032328,-0.001298,0.002669
125,0.0,1.018445,-0.124070,0.100385,-0.001147,0.000171,0.002648,0.039852,0.001909,-0.002170
126,0.0,1.019372,-0.122745,0.099874,-0.000222,0.001574,0.002381,0.037449,-0.000080,-0.005643
127,0.0,1.021171,-0.121326,0.094987,0.001576,0.003070,-0.002270,0.028818,-0.000038,-0.001446


In [173]:
m,n,r = X_test.shape
out_arr_test = np.column_stack((np.repeat(np.arange(m),n),X_test.reshape(m*n,-1)))
out_df_test = pd.DataFrame(out_arr_test)

In [83]:
X_train[0][0:, 0]

array([1.012817, 1.022833, 1.022028, 1.017877, 1.02368 , 1.016974,
       1.017746, 1.019263, 1.016417, 1.020745, 1.018643, 1.019521,
       1.02026 , 1.018041, 1.020829, 1.018644, 1.019398, 1.020399,
       1.019222, 1.022093, 1.020433, 1.020534, 1.021503, 1.019931,
       1.02048 , 1.018945, 1.019238, 1.019989, 1.018917, 1.019762,
       1.019021, 1.017887, 1.018136, 1.019543, 1.020242, 1.018757,
       1.019534, 1.019862, 1.01906 , 1.020717, 1.021055, 1.020178,
       1.018108, 1.014776, 1.015374, 1.018429, 1.019895, 1.018647,
       1.016387, 1.017053, 1.019572, 1.021097, 1.019488, 1.017218,
       1.019876, 1.022022, 1.020574, 1.021588, 1.022298, 1.019369,
       1.01698 , 1.016774, 1.016079, 1.015292, 1.018851, 1.02238 ,
       1.020781, 1.020218, 1.021344, 1.020522, 1.01979 , 1.019216,
       1.018307, 1.017996, 1.017932, 1.018121, 1.018305, 1.018458,
       1.018201, 1.017129, 1.017814, 1.0188  , 1.017601, 1.01797 ,
       1.018489, 1.017787, 1.019167, 1.019789, 1.019462, 1.020

In [371]:
funcs = ['std','median','min', 'max',
        'skew', pd.DataFrame.kurt]

df = out_df.groupby(0).agg({1:funcs}).join(out_df.groupby(0).agg({2:funcs}))
for i in range(3, 10):
    df = df.join(out_df.groupby(0).agg({i:funcs}))
    
df_test = out_df_test.groupby(0).agg({1:funcs}).join(out_df_test.groupby(0).agg({2:funcs}))
for i in range(3, 10):
    df_test = df_test.join(out_df_test.groupby(0).agg({i:funcs}))
    
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(df)
X_test_sc = scaler.transform(df_test)

y_train_ = y_train.argmax(axis=1)
y_test_ = y_test.argmax(axis=1)


In [373]:
kf = KFold(n_splits=5)

In [381]:


# reg = LogisticRegression(max_iter=1000, random_state=42, C=2, multi_class='multinomial')
reg = LogisticRegression(max_iter=1500, random_state=42, multi_class='multinomial')
params = {'C': [4, 6, 8, 10, 12]}
reg_CV = GridSearchCV(reg, params, cv=kf, verbose=10)
reg_CV.fit(X_train_sc, y_train_)
pred = reg_CV.predict(X_test_sc)
multiclass_roc_auc_score(y_test_, pred)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START C=4.........................................................
[CV 1/5; 1/4] END .......................................C=4; total time=   1.9s
[CV 2/5; 1/4] START C=4.........................................................
[CV 2/5; 1/4] END .......................................C=4; total time=   1.9s
[CV 3/5; 1/4] START C=4.........................................................
[CV 3/5; 1/4] END .......................................C=4; total time=   2.0s
[CV 4/5; 1/4] START C=4.........................................................
[CV 4/5; 1/4] END .......................................C=4; total time=   2.2s
[CV 5/5; 1/4] START C=4.........................................................
[CV 5/5; 1/4] END .......................................C=4; total time=   2.0s
[CV 1/5; 2/4] START C=6.........................................................
[CV 1/5; 2/4] END ...............................

0.9197828299966068

In [382]:
reg_CV.best_params_

{'C': 10}

In [209]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average='micro')

0.896776382762131

In [180]:
from sklearn.metrics import roc_auc_score

In [395]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

svc = OneVsRestClassifier(LinearSVC(max_iter=10000, random_state=42))
params = {'C': [8, 10, 12]}
svc_CV = GridSearchCV(reg, params, cv=kf, verbose=10).fit(X_train_sc, y_train_)
pred_svc = clf.predict(X_test_sc)
multiclass_roc_auc_score(y_test_, pred_svm)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5; 1/3] START C=8.........................................................
[CV 1/5; 1/3] END .......................................C=8; total time=   2.6s
[CV 2/5; 1/3] START C=8.........................................................
[CV 2/5; 1/3] END .......................................C=8; total time=   2.8s
[CV 3/5; 1/3] START C=8.........................................................
[CV 3/5; 1/3] END .......................................C=8; total time=   2.5s
[CV 4/5; 1/3] START C=8.........................................................
[CV 4/5; 1/3] END .......................................C=8; total time=   3.4s
[CV 5/5; 1/3] START C=8.........................................................
[CV 5/5; 1/3] END .......................................C=8; total time=   2.7s
[CV 1/5; 2/3] START C=10........................................................
[CV 1/5; 2/3] END ...............................

0.9218187987784189

In [396]:
svc_CV.best_params_

{'C': 10}

In [398]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(criterion='entropy', random_state=42)
params = {'n_estimators': [10, 20, 50, 75], 'max_depth': [2, 3, 4, 5, 6]}
clf_rfc =  GridSearchCV(rfc, params, cv=kf, verbose=10)
clf_rfc.fit(X_train_sc, y_train_)
pred_rfc = clf_rfc.predict(X_test_sc)
multiclass_roc_auc_score(y_test_, pred_rfc)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5; 1/20] START max_depth=2, n_estimators=10...............................
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elap

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.
[CV 1/5; 1/20] END .............max_depth=2, n_estimators=10; total time=   0.1s
[CV 2/5; 1/20] START max_depth=2, n_estimators=10...............................
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elaps

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.
[CV 2/5; 1/20] END .............max_depth=2, n_estimators=10; total time=   0.1s
[CV 3/5; 1/20] START max_depth=2, n_estimators=10...............................
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


...

building tree 7 of 75
building tree 8 of 75
building tree 9 of 75
building tree 10 of 75
building tree 11 of 75


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.3s remaining:    0.0s


building tree 12 of 75
building tree 13 of 75
building tree 14 of 75
building tree 15 of 75
building tree 16 of 75
building tree 17 of 75
building tree 18 of 75
building tree 19 of 75
building tree 20 of 75
building tree 21 of 75
building tree 22 of 75
building tree 23 of 75
building tree 24 of 75
building tree 25 of 75
building tree 26 of 75
building tree 27 of 75
building tree 28 of 75
building tree 29 of 75
building tree 30 of 75
building tree 31 of 75
building tree 32 of 75
building tree 33 of 75
building tree 34 of 75
building tree 35 of 75
building tree 36 of 75
building tree 37 of 75
building tree 38 of 75
building tree 39 of 75
building tree 40 of 75
building tree 41 of 75
building tree 42 of 75
building tree 43 of 75
building tree 44 of 75
building tree 45 of 75
building tree 46 of 75
building tree 47 of 75
building tree 48 of 75
building tree 49 of 75
building tree 50 of 75
building tree 51 of 75
building tree 52 of 75
building tree 53 of 75
building tree 54 of 75
building tr

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.0s finished


0.5009840515778757

In [399]:
clf_rfc.best_params_

{'max_depth': 6, 'n_estimators': 75}

In [403]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(learning_rate = 'adaptive', tol=1e-5, random_state=42)
params = {'hidden_layer_sizes': [(10,), (25,), (50,), (100,)], 
          'activation': ['rele,' 'tanh', 'logistic'], 
          'alpha': [0.05, 0.1, 0.5, 1],
          'max_iter': [100, 200, 300]}
clf_MLP =  GridSearchCV(MLP, params, cv=kf, verbose=10)
clf_MLP.fit(X_train_sc, y_train_)
pred_MLP = clf_MLP.predict(X_test_sc)
round(multiclass_roc_auc_score(y_test_, pred_MLP), 3)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5; 1/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100
[CV 1/5; 1/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100; total time=   0.0s
[CV 2/5; 1/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100
[CV 2/5; 1/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100; total time=   0.0s
[CV 3/5; 1/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100
[CV 3/5; 1/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100; total time=   0.0s
[CV 4/5; 1/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100
[CV 4/5; 1/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100; total time=   0.0s
[CV 5/5; 1/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(10,), max_iter=100
[CV 5/5; 1/96] END a

c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 450, in _validate_hyperparameters
    % (self.activation, list(sorted(ACTIVATIONS))))
ValueErr

c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 450, in _validate_hyperparameters
    % (self.activation, list(sorted(ACTIVATIONS))))
ValueErr

[CV 1/5; 12/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300; total time=   0.0s
[CV 2/5; 12/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300
[CV 2/5; 12/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300; total time=   0.0s
[CV 3/5; 12/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300
[CV 3/5; 12/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300; total time=   0.0s
[CV 4/5; 12/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300
[CV 4/5; 12/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300; total time=   0.0s
[CV 5/5; 12/96] START activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300
[CV 5/5; 12/96] END activation=rele,tanh, alpha=0.05, hidden_layer_sizes=(100,), max_iter=300; total time=   0.0s
[CV 1/5; 13/96] START activation=rele,tanh, al

c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 450, in _validate_hyperparameters
    % (self.activation, list(sorted(ACTIVATIONS))))
ValueErr

...

[CV 4/5; 94/96] END activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=100; total time=   3.3s
[CV 5/5; 94/96] START activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=100


c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV 5/5; 94/96] END activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=100; total time=   3.4s
[CV 1/5; 95/96] START activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200
[CV 1/5; 95/96] END activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200; total time=   6.5s
[CV 2/5; 95/96] START activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200
[CV 2/5; 95/96] END activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200; total time=   4.2s
[CV 3/5; 95/96] START activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200
[CV 3/5; 95/96] END activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200; total time=   4.2s
[CV 4/5; 95/96] START activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200
[CV 4/5; 95/96] END activation=logistic, alpha=1, hidden_layer_sizes=(100,), max_iter=200; total time=   5.7s
[CV 5/5; 95/96] START activation=logistic, alpha=1, hidden_layer_sizes=(100,), max

c:\users\arcry\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.85037972 0.85949287 0.86085323 0.86493394 0.87132822 0.87377553
 0.8596305  0.86343937 0.86643118 0.86847237 0.8754092  0.87486499
 0.84915505 0.8571817  0.85826912 0.86384523 0.87064832 0.87241536
 0.8585416  0.86044729 0.86588734 0.86629587 0.87323261 0.87364031
 0.84071967 0.84167196 0.84208031 0.849017

0.914

In [404]:
clf_MLP.best_params_

{'activation': 'logistic',
 'alpha': 0.05,
 'hidden_layer_sizes': (100,),
 'max_iter': 200}

In [366]:
from catboost import CatBoostClassifier
clf_cat = CatBoostClassifier(iterations=2000,
                           random_state=42,
                           learning_rate=0.5,
                           depth=3,
                           loss_function='MultiClass')
clf_cat.fit(X_train_sc, y_train_)
pred_cat = clf_cat.predict(X_test_sc)
multiclass_roc_auc_score(y_test_, pred_cat)


0:	learn: 0.9437768	total: 18.9ms	remaining: 37.8s
1:	learn: 0.6796141	total: 33ms	remaining: 32.9s
2:	learn: 0.5436701	total: 47.3ms	remaining: 31.5s
3:	learn: 0.4701266	total: 61.8ms	remaining: 30.8s
4:	learn: 0.4399633	total: 74.7ms	remaining: 29.8s
5:	learn: 0.3930672	total: 95.3ms	remaining: 31.7s
6:	learn: 0.3639381	total: 110ms	remaining: 31.4s
7:	learn: 0.3390808	total: 124ms	remaining: 30.9s
8:	learn: 0.3141234	total: 137ms	remaining: 30.4s
9:	learn: 0.2977008	total: 150ms	remaining: 29.9s
10:	learn: 0.2833582	total: 188ms	remaining: 34.1s
11:	learn: 0.2742852	total: 203ms	remaining: 33.6s
12:	learn: 0.2657333	total: 219ms	remaining: 33.5s
13:	learn: 0.2560948	total: 236ms	remaining: 33.4s
14:	learn: 0.2436708	total: 249ms	remaining: 33s
15:	learn: 0.2367961	total: 262ms	remaining: 32.5s
16:	learn: 0.2282767	total: 276ms	remaining: 32.2s
17:	learn: 0.2203085	total: 294ms	remaining: 32.4s
18:	learn: 0.2137680	total: 326ms	remaining: 34s
19:	learn: 0.2108920	total: 353ms	remaini

168:	learn: 0.0348368	total: 2.43s	remaining: 26.4s
169:	learn: 0.0345362	total: 2.45s	remaining: 26.3s
170:	learn: 0.0344450	total: 2.46s	remaining: 26.3s
171:	learn: 0.0343235	total: 2.47s	remaining: 26.2s
172:	learn: 0.0341459	total: 2.48s	remaining: 26.2s
173:	learn: 0.0339063	total: 2.49s	remaining: 26.1s
174:	learn: 0.0336142	total: 2.5s	remaining: 26.1s
175:	learn: 0.0334485	total: 2.51s	remaining: 26s
176:	learn: 0.0332965	total: 2.52s	remaining: 26s
177:	learn: 0.0331390	total: 2.53s	remaining: 25.9s
178:	learn: 0.0329395	total: 2.54s	remaining: 25.8s
179:	learn: 0.0328097	total: 2.55s	remaining: 25.8s
180:	learn: 0.0325829	total: 2.56s	remaining: 25.7s
181:	learn: 0.0324867	total: 2.57s	remaining: 25.7s
182:	learn: 0.0322868	total: 2.58s	remaining: 25.6s
183:	learn: 0.0318912	total: 2.59s	remaining: 25.6s
184:	learn: 0.0315875	total: 2.61s	remaining: 25.6s
185:	learn: 0.0313862	total: 2.62s	remaining: 25.6s
186:	learn: 0.0307040	total: 2.63s	remaining: 25.5s
187:	learn: 0.030

337:	learn: 0.0136002	total: 5.07s	remaining: 24.9s
338:	learn: 0.0135241	total: 5.08s	remaining: 24.9s
339:	learn: 0.0134334	total: 5.09s	remaining: 24.9s
340:	learn: 0.0134049	total: 5.11s	remaining: 24.8s
341:	learn: 0.0133333	total: 5.13s	remaining: 24.8s
342:	learn: 0.0133257	total: 5.14s	remaining: 24.9s
343:	learn: 0.0132798	total: 5.16s	remaining: 24.8s
344:	learn: 0.0132510	total: 5.18s	remaining: 24.9s
345:	learn: 0.0132147	total: 5.2s	remaining: 24.8s
346:	learn: 0.0131747	total: 5.21s	remaining: 24.8s
347:	learn: 0.0131501	total: 5.23s	remaining: 24.8s
348:	learn: 0.0130721	total: 5.24s	remaining: 24.8s
349:	learn: 0.0129992	total: 5.25s	remaining: 24.8s
350:	learn: 0.0129757	total: 5.27s	remaining: 24.8s
351:	learn: 0.0129126	total: 5.28s	remaining: 24.7s
352:	learn: 0.0128229	total: 5.3s	remaining: 24.7s
353:	learn: 0.0128038	total: 5.31s	remaining: 24.7s
354:	learn: 0.0127826	total: 5.33s	remaining: 24.7s
355:	learn: 0.0127644	total: 5.34s	remaining: 24.7s
356:	learn: 0.

498:	learn: 0.0078719	total: 7.09s	remaining: 21.3s
499:	learn: 0.0078657	total: 7.1s	remaining: 21.3s
500:	learn: 0.0078537	total: 7.11s	remaining: 21.3s
501:	learn: 0.0078286	total: 7.12s	remaining: 21.3s
502:	learn: 0.0077555	total: 7.13s	remaining: 21.2s
503:	learn: 0.0077513	total: 7.14s	remaining: 21.2s
504:	learn: 0.0077377	total: 7.16s	remaining: 21.2s
505:	learn: 0.0077344	total: 7.17s	remaining: 21.2s
506:	learn: 0.0077096	total: 7.18s	remaining: 21.1s
507:	learn: 0.0076945	total: 7.19s	remaining: 21.1s
508:	learn: 0.0076660	total: 7.21s	remaining: 21.1s
509:	learn: 0.0075991	total: 7.22s	remaining: 21.1s
510:	learn: 0.0075899	total: 7.23s	remaining: 21.1s
511:	learn: 0.0075690	total: 7.24s	remaining: 21.1s
512:	learn: 0.0075378	total: 7.26s	remaining: 21s
513:	learn: 0.0074807	total: 7.28s	remaining: 21s
514:	learn: 0.0074664	total: 7.29s	remaining: 21s
515:	learn: 0.0074616	total: 7.31s	remaining: 21s
516:	learn: 0.0074524	total: 7.36s	remaining: 21.1s
517:	learn: 0.0074449

658:	learn: 0.0052045	total: 8.92s	remaining: 18.1s
659:	learn: 0.0052009	total: 8.93s	remaining: 18.1s
660:	learn: 0.0051990	total: 8.94s	remaining: 18.1s
661:	learn: 0.0051733	total: 8.95s	remaining: 18.1s
662:	learn: 0.0051515	total: 8.96s	remaining: 18.1s
663:	learn: 0.0051425	total: 8.97s	remaining: 18s
664:	learn: 0.0051307	total: 8.97s	remaining: 18s
665:	learn: 0.0051161	total: 8.98s	remaining: 18s
666:	learn: 0.0051053	total: 8.99s	remaining: 18s
667:	learn: 0.0050772	total: 9.01s	remaining: 18s
668:	learn: 0.0050637	total: 9.02s	remaining: 17.9s
669:	learn: 0.0050572	total: 9.03s	remaining: 17.9s
670:	learn: 0.0050552	total: 9.04s	remaining: 17.9s
671:	learn: 0.0050520	total: 9.05s	remaining: 17.9s
672:	learn: 0.0050364	total: 9.06s	remaining: 17.9s
673:	learn: 0.0050123	total: 9.07s	remaining: 17.8s
674:	learn: 0.0050032	total: 9.08s	remaining: 17.8s
675:	learn: 0.0049931	total: 9.1s	remaining: 17.8s
676:	learn: 0.0049812	total: 9.11s	remaining: 17.8s
677:	learn: 0.0049729	t

...

1488:	learn: 0.0014907	total: 19s	remaining: 6.51s
1489:	learn: 0.0014892	total: 19s	remaining: 6.5s
1490:	learn: 0.0014891	total: 19s	remaining: 6.49s
1491:	learn: 0.0014888	total: 19s	remaining: 6.47s
1492:	learn: 0.0014863	total: 19s	remaining: 6.46s
1493:	learn: 0.0014839	total: 19s	remaining: 6.45s
1494:	learn: 0.0014835	total: 19.1s	remaining: 6.44s
1495:	learn: 0.0014827	total: 19.1s	remaining: 6.42s
1496:	learn: 0.0014801	total: 19.1s	remaining: 6.41s
1497:	learn: 0.0014799	total: 19.1s	remaining: 6.4s
1498:	learn: 0.0014787	total: 19.1s	remaining: 6.38s
1499:	learn: 0.0014778	total: 19.1s	remaining: 6.37s
1500:	learn: 0.0014760	total: 19.1s	remaining: 6.36s
1501:	learn: 0.0014758	total: 19.1s	remaining: 6.34s
1502:	learn: 0.0014749	total: 19.1s	remaining: 6.33s
1503:	learn: 0.0014745	total: 19.2s	remaining: 6.32s
1504:	learn: 0.0014739	total: 19.2s	remaining: 6.3s
1505:	learn: 0.0014719	total: 19.2s	remaining: 6.29s
1506:	learn: 0.0014701	total: 19.2s	remaining: 6.28s
1507:	le

1647:	learn: 0.0012643	total: 20.8s	remaining: 4.45s
1648:	learn: 0.0012640	total: 20.8s	remaining: 4.43s
1649:	learn: 0.0012605	total: 20.8s	remaining: 4.42s
1650:	learn: 0.0012598	total: 20.9s	remaining: 4.41s
1651:	learn: 0.0012581	total: 20.9s	remaining: 4.4s
1652:	learn: 0.0012573	total: 20.9s	remaining: 4.38s
1653:	learn: 0.0012552	total: 20.9s	remaining: 4.37s
1654:	learn: 0.0012548	total: 20.9s	remaining: 4.36s
1655:	learn: 0.0012540	total: 20.9s	remaining: 4.34s
1656:	learn: 0.0012538	total: 20.9s	remaining: 4.33s
1657:	learn: 0.0012536	total: 20.9s	remaining: 4.32s
1658:	learn: 0.0012527	total: 21s	remaining: 4.31s
1659:	learn: 0.0012520	total: 21s	remaining: 4.29s
1660:	learn: 0.0012519	total: 21s	remaining: 4.28s
1661:	learn: 0.0012504	total: 21s	remaining: 4.27s
1662:	learn: 0.0012499	total: 21s	remaining: 4.26s
1663:	learn: 0.0012494	total: 21s	remaining: 4.24s
1664:	learn: 0.0012480	total: 21s	remaining: 4.23s
1665:	learn: 0.0012474	total: 21s	remaining: 4.22s
1666:	lear

1808:	learn: 0.0011196	total: 22.8s	remaining: 2.41s
1809:	learn: 0.0011188	total: 22.8s	remaining: 2.4s
1810:	learn: 0.0011170	total: 22.8s	remaining: 2.38s
1811:	learn: 0.0011160	total: 22.9s	remaining: 2.37s
1812:	learn: 0.0011159	total: 22.9s	remaining: 2.36s
1813:	learn: 0.0011128	total: 22.9s	remaining: 2.35s
1814:	learn: 0.0011119	total: 22.9s	remaining: 2.33s
1815:	learn: 0.0011115	total: 22.9s	remaining: 2.32s
1816:	learn: 0.0011106	total: 22.9s	remaining: 2.31s
1817:	learn: 0.0011101	total: 22.9s	remaining: 2.29s
1818:	learn: 0.0011098	total: 22.9s	remaining: 2.28s
1819:	learn: 0.0011084	total: 22.9s	remaining: 2.27s
1820:	learn: 0.0011082	total: 23s	remaining: 2.26s
1821:	learn: 0.0011071	total: 23s	remaining: 2.24s
1822:	learn: 0.0011068	total: 23s	remaining: 2.23s
1823:	learn: 0.0011043	total: 23s	remaining: 2.22s
1824:	learn: 0.0011039	total: 23s	remaining: 2.2s
1825:	learn: 0.0011034	total: 23s	remaining: 2.19s
1826:	learn: 0.0011028	total: 23s	remaining: 2.18s
1827:	lea

1964:	learn: 0.0009979	total: 24.5s	remaining: 436ms
1965:	learn: 0.0009972	total: 24.5s	remaining: 423ms
1966:	learn: 0.0009963	total: 24.5s	remaining: 411ms
1967:	learn: 0.0009956	total: 24.5s	remaining: 398ms
1968:	learn: 0.0009953	total: 24.5s	remaining: 386ms
1969:	learn: 0.0009949	total: 24.5s	remaining: 373ms
1970:	learn: 0.0009947	total: 24.5s	remaining: 361ms
1971:	learn: 0.0009937	total: 24.5s	remaining: 348ms
1972:	learn: 0.0009934	total: 24.5s	remaining: 336ms
1973:	learn: 0.0009933	total: 24.6s	remaining: 323ms
1974:	learn: 0.0009922	total: 24.6s	remaining: 311ms
1975:	learn: 0.0009921	total: 24.6s	remaining: 299ms
1976:	learn: 0.0009903	total: 24.6s	remaining: 286ms
1977:	learn: 0.0009886	total: 24.6s	remaining: 274ms
1978:	learn: 0.0009884	total: 24.6s	remaining: 261ms
1979:	learn: 0.0009880	total: 24.6s	remaining: 249ms
1980:	learn: 0.0009876	total: 24.6s	remaining: 236ms
1981:	learn: 0.0009874	total: 24.6s	remaining: 224ms
1982:	learn: 0.0009866	total: 24.7s	remaining:

0.9228367831693248